In [20]:
import simplekml
import polyline
import csv

def create_kml(csv_file, output_kml, columns_mapping):
    kml = simplekml.Kml()
    
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            name = row['name']
            polyline_str = row['map.polyline']
            comp = int(row['competitive'])
            
            # Extract additional columns for the description
            additional_info = ""
            for key, display_name in columns_mapping.items():
                if key == 'id':  # Check if the column is 'More info'
                    additional_info += f"<tr><td>{display_name}</td><td>https://www.strava.com/segments/{row[key]}</td></tr>"
                else:
                    additional_info += f"<tr><td>{display_name}</td><td>{row[key]}</td></tr>"
            
            coordinates = polyline.decode(polyline_str, geojson=True)
            
            lin = kml.newlinestring(name=name, coords=coordinates)
            lin.description = f"<table border='1'><tr></tr>{additional_info}</table>"
            # Increase line thickness
            lin.style.linestyle.width = 2  # Adjust the value as needed
            if comp == -1:
                lin.style.linestyle.color = simplekml.Color.green
            elif comp == 0:
                lin.style.linestyle.color = simplekml.Color.blue
            else:
                lin.style.linestyle.color = simplekml.Color.red
    
    kml.save(output_kml)

# Example usage:
columns_mapping = {'total_elevation_gain': 'Elevation Gain (m)',
                  'xoms.overall':'Fastest Time (H:MM:SS)', 'athlete_count':'# Athletes that have completed'
                  ,'id':'More info'}  # Specify the columns and their display names
create_kml('strava_comp.csv', 'output.kml', columns_mapping)
